In [1]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense,Dropout
import numpy as np

import random

data = []
raw_vocab = set()
meta = []
width = 5
height = 5
for ii in range(2000):
    width = random.randint(0,2)*2+5
    height = random.randint(0,2)*2+5
    
    
    north_door = random.random() < 0.3
    south_door = random.random() < 0.3
    east_door = random.random() < 0.3
    west_door = random.random() < 0.3
    
    if not (north_door or south_door or east_door or west_door):
        door = random.choice(['n','s','e','w'])
        north_door = door == 'n'
        south_door = door == 's'
        east_door = door == 'e'
        west_door = door == 'w'
    
    stuff = list('.'*((width-2)*(height-2)))
    if random.random() < 0.3:
        stuff[0] = 'E'
    if random.random() < 0.3:
        stuff[1] = '/'
    if random.random() < 0.8:
        stuff[2] = 'I'
        
    random.shuffle(stuff)
    
    output = [list('#'*width) for ii in range(height)]
    if north_door:
        output[0][width/2] = '-'
    if south_door:
        output[-1][width/2] = '-'
    if east_door:
        output[height/2][0] = '|'
    if west_door:
        output[height/2][-1] = '|'
    
    ind = 0
    for yy in range(1,height-1):
        for xx in range(1,width-1):
            output[yy][xx] = stuff[ind]
            ind += 1
            
    data.append('{\n' + '\n'.join([''.join(row) for row in output]) +'\n}')
    meta.append((stuff.count('E'),stuff.count('/'),stuff.count('I'),north_door,south_door,east_door,west_door,width,height))
    raw_vocab |= set(data[-1])    

Using TensorFlow backend.


In [27]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
lstm_dim = 128  # Latent dimensionality of the encoding space.
num_samples = len(data)  # Number of samples to train on.
latent_size = 32



input_characters = sorted(list(raw_vocab))
target_characters = sorted(list(raw_vocab))
num_encoder_tokens = len(raw_vocab)
num_decoder_tokens = len(raw_vocab)
max_encoder_seq_length = max([len(txt) for txt in data])
max_decoder_seq_length = max([len(txt) for txt in data])

print('Number of samples:', len(data))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(data), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(data), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(data), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(data, data)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

            
            

Number of samples: 2000
Number of unique input tokens: 10
Number of unique output tokens: 10
Max sequence length for inputs: 93
Max sequence length for outputs: 93


In [28]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(lstm_dim, return_state=True)
encoder_outputs, state_h_, state_c_ = encoder(encoder_inputs)

dropout = 0.5


latent_h = Dropout(dropout)(latent_h)
latent_c = Dropout(dropout)(latent_c)

latent_squeeze = Dense(latent_size,activation='tanh')
latent_h = latent_squeeze(state_h_)
latent_c = latent_squeeze(state_c_)

latent_stretch = Dense(lstm_dim)
state_h = latent_stretch(latent_h)
state_c = latent_stretch(latent_c)


# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(lstm_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)


decoder_outputs = Dropout(dropout)(decoder_outputs)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
from keras import optimizers
model.compile(optimizer=optimizers.RMSprop( decay=1e-6), loss='categorical_crossentropy',
              metrics=['accuracy'])

In [31]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=5000,
          validation_split=0.2)

Train on 1600 samples, validate on 400 samples
Epoch 1/5000
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0863 - acc: 0.6285 - val_loss: 0.0900 - val_acc: 0.6341
Epoch 2/5000
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0871 - acc: 0.6301 - val_loss: 0.0894 - val_acc: 0.6408
Epoch 3/5000
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0860 - acc: 0.6361 - val_loss: 0.0885 - val_acc: 0.6507
Epoch 4/5000
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0841 - acc: 0.6374 - val_loss: 0.0979 - val_acc: 0.6472
Epoch 5/5000
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0867 - acc: 0.6370 - val_loss: 0.0883 - val_acc: 0.6527
Epoch 6/5000
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0857 - acc: 0.6416 - val_loss: 0.0926 - val_acc: 0.6490
Epoch 7/5000
1600/1600 [==============================] - 5s 3ms/step - loss: 0.0845 - acc: 0.6401 - val_loss: 0.0993 - val_acc: 0.6466
E

KeyboardInterrupt: 

In [32]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models



encoder_model_2_latent = Model(encoder_inputs, [latent_h,latent_c])

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(lstm_dim,))
decoder_state_input_c = Input(shape=(lstm_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


model.save('full_{}_{}_.model'.format(lstm_dim,latent_size))
encoder_model.save('encoder_{}_{}_.model'.format(lstm_dim,latent_size))
decoder_model.save('decoder_{}_{}_.model'.format(lstm_dim,latent_size))


# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    latent =  encoder_model_2_latent.predict(input_seq)
    print(latent)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['{']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = sample(output_tokens[0, -1, :],temperature=1.0)
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '}' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

from sklearn import linear_model



print(encoder_input_data.shape)
X = encoder_model_2_latent.predict(encoder_input_data)
X = np.hstack([X[0],X[1]])
y = np.array(meta)
clf = linear_model.Lasso(alpha=0.1)
for ii in range(y.shape[1]):
    print(ii)
    clf.fit(X,y[:,ii])
    print(clf.coef_)
    print(clf.score(X,y[:,ii]))

for seq_index in range(len(data)):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]


    latent_h_,latent_c_ = encoder_model_2_latent.predict(input_seq)
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', data[seq_index])
    print('Decoded sentence:', decoded_sentence)

/usr/local/lib/python2.7/dist-packages/keras/engine/topology.py:2344: UserWarning: Layer lstm_16 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'input_29:0' shape=(?, 128) dtype=float32>, <tf.Tensor 'input_30:0' shape=(?, 128) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


(2000, 93, 10)
0
[ 0.  0. -0. -0. -0. -0. -0. -0.  0. -0.  0.  0. -0. -0.  0.  0.  0. -0.
  0.  0.  0. -0. -0.  0.  0. -0.  0.  0.  0. -0. -0. -0.  0.  0.  0. -0.
 -0.  0.  0. -0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0. -0.  0. -0.
  0.  0. -0.  0.  0.  0. -0. -0.  0. -0.]
-2.220446049250313e-16
1
[-0.  0.  0.  0.  0. -0. -0.  0. -0.  0.  0. -0. -0. -0.  0. -0. -0. -0.
  0. -0. -0. -0.  0. -0.  0. -0. -0.  0.  0.  0. -0. -0. -0.  0. -0. -0.
  0. -0. -0.  0. -0.  0. -0. -0. -0. -0.  0. -0. -0. -0. -0.  0. -0. -0.
 -0.  0.  0. -0. -0.  0. -0.  0.  0. -0.]
-2.220446049250313e-16
2
[ 0.  0. -0.  0. -0. -0.  0.  0.  0.  0. -0.  0. -0. -0.  0. -0. -0.  0.
  0. -0. -0. -0. -0.  0. -0.  0. -0. -0. -0.  0. -0.  0. -0. -0. -0.  0.
 -0.  0.  0. -0.  0.  0. -0. -0.  0. -0. -0.  0.  0. -0.  0. -0. -0.  0.
 -0.  0. -0.  0. -0. -0.  0. -0. -0.  0.]
-3.774758283725532e-15
3
[ 0.         -0.          0.          0.         -0.          0.
  0.          0.          0.          0.01508544 -0.03704758


KeyboardInterrupt



In [17]:
print(meta[0])
print 

(0, 0, 1, True, False, False, False, 5, 5)


<function print>